In [2]:
library("scRNAseq")
library(scater)

data <- readRDS('data_NGS.rds')

Loading required package: SingleCellExperiment
Loading required package: SummarizedExperiment
Warning message:
"package 'SummarizedExperiment' was built under R version 3.6.2"Loading required package: GenomicRanges
Loading required package: stats4
Loading required package: BiocGenerics
Loading required package: parallel

Attaching package: 'BiocGenerics'

The following objects are masked from 'package:parallel':

    clusterApply, clusterApplyLB, clusterCall, clusterEvalQ,
    clusterExport, clusterMap, parApply, parCapply, parLapply,
    parLapplyLB, parRapply, parSapply, parSapplyLB

The following objects are masked from 'package:stats':

    IQR, mad, sd, var, xtabs

The following objects are masked from 'package:base':

    anyDuplicated, append, as.data.frame, basename, cbind, colnames,
    dirname, do.call, duplicated, eval, evalq, Filter, Find, get, grep,
    grepl, intersect, is.unsorted, lapply, Map, mapply, match, mget,
    order, paste, pmax, pmax.int, pmin, pmin.int, Positi

In [4]:
install.packages("BiocManager")
BiocManager::install("multtest")
install.packages("Seurat")

package 'BiocManager' successfully unpacked and MD5 sums checked

The downloaded binary packages are in
	C:\Users\marze\AppData\Local\Temp\RtmpYRdxLr\downloaded_packages


Bioconductor version 3.10 (BiocManager 1.30.10), R 3.6.1 (2019-07-05)
Installing package(s) 'multtest'


package 'multtest' successfully unpacked and MD5 sums checked

The downloaded binary packages are in
	C:\Users\marze\AppData\Local\Temp\RtmpYRdxLr\downloaded_packages


Old packages: 'agricolae', 'ape', 'askpass', 'backports', 'BH', 'boot',
  'broom', 'callr', 'caret', 'class', 'cli', 'clipr', 'cluster', 'curl',
  'data.table', 'dbplyr', 'digest', 'dplyr', 'emmeans', 'evaluate', 'fansi',
  'forcats', 'foreach', 'formatR', 'fs', 'ggplot2', 'glmnet', 'glue', 'gower',
  'haven', 'hexbin', 'hms', 'htmltools', 'htmlwidgets', 'httpuv', 'httr',
  'ipred', 'IRkernel', 'iterators', 'jsonlite', 'KernSmooth', 'knitr',
  'labelled', 'later', 'lattice', 'lava', 'lubridate', 'markdown', 'MASS',
  'Matrix', 'mgcv', 'mime', 'ModelMetrics', 'modelr', 'nlme', 'nnet',
  'numDeriv', 'openssl', 'pillar', 'pkgconfig', 'plyr', 'prettyunits',
  'processx', 'prodlim', 'progress', 'promises', 'ps', 'purrr', 'quantmod',
  'questionr', 'R6', 'Rcpp', 'recipes', 'repr', 'reprex', 'reshape2',
  'rmarkdown', 'rstudioapi', 'rvest', 'scales', 'scater', 'selectr', 'shiny',
  'sp', 'spatial', 'SQUAREM', 'stringi', 'survival', 'sys', 'tibble', 'tidyr',
  'tidyselect', 'tidyverse', 'tinyt

package 'Seurat' successfully unpacked and MD5 sums checked

The downloaded binary packages are in
	C:\Users\marze\AppData\Local\Temp\RtmpYRdxLr\downloaded_packages


In [ ]:
library("Seurat")

Warning message:
"package 'Seurat' was built under R version 3.6.3"

In [3]:
version

               _                           
platform       x86_64-w64-mingw32          
arch           x86_64                      
os             mingw32                     
system         x86_64, mingw32             
status                                     
major          3                           
minor          6.1                         
year           2019                        
month          07                          
day            05                          
svn rev        76782                       
language       R                           
version.string R version 3.6.1 (2019-07-05)
nickname       Action of the Toes          

In [3]:
#Analiza jakości na poziomie komórki
cellQC <- perCellQCMetrics(data)
head(cellQC)

DataFrame with 6 rows and 7 columns
        sum  detected   percent_top_50  percent_top_100  percent_top_200
  <numeric> <integer>        <numeric>        <numeric>        <numeric>
1    641208      6948 53.0247595164128 60.6531733852354 69.5451397986301
2   1334892      8115 25.2438399510972 31.0849866506054 39.8599287432991
3   1039253      9576  22.914920620869 28.8369627030184 36.9552938504868
4   1080500      8207  28.608144377603  34.768533086534 43.8335030078667
5    540210      7419 40.4872179337665  49.055922696729 59.6468040206586
6   1460073      7217  35.945257531644 42.4046605888884 51.1084719736616
   percent_top_500     total
         <numeric> <numeric>
1 81.3222854362391    641208
2 56.9344935770085   1334892
3 51.8562852356452   1039253
4 61.4753354928274   1080500
5 74.6567075766831    540210
6 66.9472005851762   1460073

In [ ]:
colData(data) <- cbind(colData(data), cellQC) #Dodanie do kolumn 

In [ ]:
plotColData(data, x = "sum", y = "detected", theme_size = 20, point_size = 3)

In [ ]:
cellQCLogical <- quickPerCellQC(cellQC)

In [ ]:
head(cellQCLogical)

In [ ]:
lowQuality <- isOutlier(colData(data)$sum , nmads = 3,type = "lower", log = TRUE)
badCells <- data[,lowQuality ]
head(badCells)

In [ ]:
colData(badCells)$title

In [ ]:
data[colData(data)$title!=colData(badCells)$title]

In [ ]:
new_data <- data[colData(data)$title!=colData(badCells)$title]
new_data

In [ ]:
#3
featureQC <- perFeatureQCMetrics(data) #Analiza na poziomie genów

In [ ]:
head(featureQC)

In [ ]:
featureQC[featureQC$detected!=0,]

In [ ]:
dim(featureQC[featureQC$detected==0,])

In [ ]:
new_featureQC <- featureQC[featureQC$detected!=0,]

In [ ]:
plotHighestExprs(data, exprs_values="tpm", n = 5) + theme(text=element_text(size=18))

In [ ]:
#4 
# seuratObj <- as.Seurat(new_data, assay="counts",data=colData)

In [ ]:
# seuratObj <- AddMetaData(seuratObj, as.data.frame(colData(data)))
# seuratObj <- FindVariableFeatures(seuratObj)
# seuratObj <- NormalizeData(object=seuratObj,normalization.method="LogNormalize")
# seuratObj <- ScaleData(object=seuratObj) # vars .to. regress = c(" batch ")
# seuratObj <- RunPCA(object=seuratObj, npcs=30)
# # seuratObj <- RunTSNE(object=seuratObj)
# seuratObj <- FindNeighbors(seuratObj, reduction = "pca", dims = 1:30)
# seuratObj <- FindClusters(seuratObj, resolution = 0.5, algorithm = 1) 
# seuratObj